In [1]:
import numpy as np
from torchvision import datasets, transforms
import torch.nn as nn
import torch
from math import floor
from sklearn.model_selection import train_test_split
from torch.utils.data import random_split, SubsetRandomSampler
import torch.nn.functional as F
from torchsummary import summary
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [2]:
def read_pdb(filename):
    with open(filename, 'r') as file:
        strline_L = file.readlines()
        # print(strline_L)

    X_list = list()
    Y_list = list()
    Z_list = list()
    atomtype_list = list()
    for strline in strline_L:
        # removes all whitespace at the start and end, including spaces, tabs, newlines and carriage returns
        stripped_line = strline.strip()

        line_length = len(stripped_line)
        # print("Line length:{}".format(line_length))
        if line_length < 78:
            print("ERROR: line length is different. Expected>=78, current={}".format(line_length))

        X_list.append(float(stripped_line[30:38].strip()))
        Y_list.append(float(stripped_line[38:46].strip()))
        Z_list.append(float(stripped_line[46:54].strip()))

        atomtype = stripped_line[76:78].strip()
        if atomtype == 'C':
            atomtype_list.append('h') # 'h' means hydrophobic
        else:
            atomtype_list.append('p') # 'p' means polar

    return X_list, Y_list, Z_list, atomtype_list

In [3]:
def create_vox(X_list,Y_list,Z_list,width):
        X0 = floor(np.mean(X_list))
        Y0 = floor(np.mean(Y_list))
        Z0 = floor(np.mean(Z_list))
        vox = np.zeros((width*2, width*2, width*2, 4))
        x_lower = X0 - width
        x_upper = X0 + width
        y_lower = Y0 - width
        y_upper = Y0 + width
        z_lower = Z0 - width
        z_upper = Z0 + width
        return x_lower,x_upper,y_lower,y_upper,z_lower,z_upper,vox
    
def convert_xyz_to_vox(x_lower,x_upper,y_lower,y_upper,z_lower,z_upper,X_list, Y_list,Z_list,atomtype_list,dim_1,dim_2,vox):

    for j in range(len(X_list)):

            x = X_list[j]
            y = Y_list[j]
            z = Z_list[j]
            atomtype = atomtype_list[j]

            if x > x_lower and x < x_upper and y > y_lower and y < y_upper and z > z_lower and z < z_upper:
                index_x = floor((x-x_lower))
                index_y = floor((y-y_lower))
                index_z = floor((z-z_lower))
                if atomtype =='h':
                    vox[index_x,index_y,index_z, dim_1] += 1
                elif atomtype =='p':
                    vox[index_x, index_y, index_z, dim_2] += 1

    return vox

def create_pos_or_neg(pos_or_neg):
    vox_list = []
    label_list = []

    for i in range(1,3001): 
        
        lig = i
        file_name = "{}_{}_cg.pdb".format(("0000" + str(lig))[-4:], "lig")
        X_list, Y_list, Z_list, atomtype_list = read_pdb('training_data/'+file_name)
        x_lower,x_upper,y_lower,y_upper,z_lower,z_upper,vox=create_vox(X_list,Y_list,Z_list,20)
        vox= convert_xyz_to_vox(x_lower,x_upper,y_lower,y_upper,z_lower,z_upper,X_list, Y_list,Z_list,atomtype_list,0,1,vox)

        
        if pos_or_neg == 'pos':
                pro = i
        else:
                pro = np.random.choice(np.delete(np.arange(1,3001),i-1))
        
        file_name = "{}_{}_cg.pdb".format(("0000" + str(pro))[-4:], "pro")
        X_list, Y_list, Z_list, atomtype_list = read_pdb('training_data/'+file_name)
        vox=convert_xyz_to_vox(x_lower,x_upper,y_lower,y_upper,z_lower,z_upper,X_list, Y_list,Z_list,atomtype_list,2,3,vox)
        
        vox_list.append(vox)
        
        if pos_or_neg == 'pos':
            label_list.append(1)
        else:
            label_list.append(0)
    return vox_list,label_list


In [140]:
pos_vox_list, pos_label_list = create_pos_or_neg('pos')
neg_vox_list, neg_label_list = create_pos_or_neg('neg')
pos_vox_list.extend(neg_vox_list)
pos_label_list.extend(neg_label_list)


In [143]:
#reshape X
X = np.array(pos_vox_list)
y = np.array(pos_label_list)
X=np.swapaxes(X,1,4)
X=np.swapaxes(X,2,4)
X=np.swapaxes(X,3,4)
X.shape

del pos_vox_list
del pos_label_list

In [144]:
dropout = 0.2

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer_1=nn.Sequential(
            nn.Conv3d(4, 32, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(), 
            nn.MaxPool3d(2,2))
        self.layer_2=nn.Sequential(
            nn.Conv3d(32, 64, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(), 
            nn.MaxPool3d(2,2))
        self.layer_3=nn.Sequential(     
            nn.Conv3d(64, 128, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(), 
            nn.MaxPool3d(2,2))
        self.layer_4=nn.Sequential(   
            nn.Conv3d(128, 256, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(), 
            nn.MaxPool3d(2,2))
        self.layer_5=nn.Sequential( 
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(),)
        self.layer_6=nn.Sequential( 
            nn.Dropout(0.2),
            nn.Linear(512, 2))
    def forward(self, x):
        #print("in",x.shape)
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.layer_5(x)
        x = self.layer_6(x)
        return x

model = Net().to(device)


In [145]:
summary(model,(4,40,40,40))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 32, 40, 40, 40]           3,488
       BatchNorm3d-2       [-1, 32, 40, 40, 40]              64
              ReLU-3       [-1, 32, 40, 40, 40]               0
         MaxPool3d-4       [-1, 32, 20, 20, 20]               0
            Conv3d-5       [-1, 64, 20, 20, 20]          55,360
       BatchNorm3d-6       [-1, 64, 20, 20, 20]             128
              ReLU-7       [-1, 64, 20, 20, 20]               0
         MaxPool3d-8       [-1, 64, 10, 10, 10]               0
            Conv3d-9      [-1, 128, 10, 10, 10]         221,312
      BatchNorm3d-10      [-1, 128, 10, 10, 10]             256
             ReLU-11      [-1, 128, 10, 10, 10]               0
        MaxPool3d-12         [-1, 128, 5, 5, 5]               0
           Conv3d-13         [-1, 256, 5, 5, 5]         884,992
      BatchNorm3d-14         [-1, 256, 

In [146]:
#load_data
X=torch.from_numpy(X).float()
y=torch.from_numpy(y).long()

traindata=[]
for i in range(0,len(X)):
    traindata.append([X[i],y[i]])

# stratified split for validation
train_idx, valid_idx= train_test_split(np.arange(len(y)), test_size=0.2, shuffle=True, stratify=y)
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

batch_size = 64
train_loader = torch.utils.data.DataLoader(traindata, batch_size=batch_size, sampler=train_sampler, num_workers=4)
valid_loader = torch.utils.data.DataLoader(traindata, batch_size=batch_size, sampler=val_sampler, num_workers=4)

In [147]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()

lr = 0.001
weight_decay=0
optimizer = optim.Adam(model.parameters(), lr=lr, 
                       betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay)
optimizer_name = 'Adam'

In [ ]:
# number of epochs to train the model
n_epochs = 30
#List to store loss to visualize
train_losslist = []
train_acclist = []
val_losslist = []
val_acclist = []

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
#### train the model ####
    correct_train= 0
    total_train = 0
    #model.train()
    for i, (inputs,labels) in enumerate(train_loader,0) :
        # move tensors to GPU if CUDA is available
        #if train_on_gpu:
        inputs, labels = inputs.cuda(), labels.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
    
        output = model(inputs)
        
        # calculate the batch loss
        loss = criterion(output, labels)
        _, predicted = torch.max(output.data, 1)
        total_train += labels.size(0)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        correct_train += (predicted == labels).sum().item()
        # update training loss
        train_loss += loss.item()
    train_acc=correct_train/total_train
##### validate the model ####

    #model.eval()
    correct= 0
    total=0
    with torch.no_grad():
        for i, (inputs,labels) in enumerate(valid_loader,0) :
            # move tensors to GPU if CUDA is available
            #if train_on_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(inputs)

            # calculate the batch loss
            loss = criterion(output, labels)
            # update average validation loss 
            valid_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            #print(output.data,_,predicted,labels)
    cur_val_acc = correct / total

        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tTrain Accuracy:{:.6f} \tValidation Accuracy:{:.6f}'.format(
        epoch, train_loss/len(train_loader), valid_loss/len(valid_loader),train_acc,cur_val_acc))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_project.h5')
        valid_loss_min = valid_loss
        
    train_losslist.append(train_loss/len(train_loader))
    train_acclist.append(train_acc)
    val_losslist.append(valid_loss/len(valid_loader))
    val_acclist.append(cur_val_acc)



In [49]:
#train_losslist
#train_acclist
#val_losslist
#val_acclist

[0.935,
 0.9408333333333333,
 0.9408333333333333,
 0.9433333333333334,
 0.9475,
 0.9566666666666667,
 0.9641666666666666,
 0.9583333333333334,
 0.9733333333333334,
 0.9675,
 0.97,
 0.9766666666666667,
 0.9791666666666666,
 0.9525,
 0.9775,
 0.9791666666666666,
 0.9766666666666667,
 0.9691666666666666,
 0.9791666666666666,
 0.9658333333333333,
 0.9725,
 0.9566666666666667,
 0.9766666666666667,
 0.9733333333333334,
 0.9783333333333334,
 0.9758333333333333,
 0.9725,
 0.9775,
 0.9766666666666667,
 0.9758333333333333]

In [ ]:
# batch size
# train_loss_diff_batchsize = pd.DataFrame(index = np.arange(len(train_loss)))
# val_loss_diff_batchsize = pd.DataFrame(index = np.arange(len(valid_loss)))

# #train_loss_diff_batchsize['batch_size_16_train'] = train_loss_batch16
# #train_loss_diff_batchsize['batch_size_32_train'] = train_loss_batch32
# train_loss_diff_batchsize['batch_size_64_train'] = train_loss_batch64

# #train_loss_diff_batchsize['batch_size_16_val'] = val_loss_batch16
# #train_loss_diff_batchsize['batch_size_32_val'] = val_loss_batch32
# train_loss_diff_batchsize['batch_size_64_val'] = val_loss_batch64

# plt.figure(figsize=(12,8))
# sns.set(font_scale=1.5) 
# sns.lineplot(data = train_loss_diff_batchsize).set(title = 'val vs train Loss', xlabel = 'epoch', ylabel='loss')

In [232]:
# Test data load 
x_lower = []
x_upper = []
y_lower = []
y_upper = []
z_lower = []
z_upper = []
vox_test_list = []

def read_pdb_test(filename):
    with open(filename,'r') as file:
        strline_L = file.readlines()
        # print(strline_L)
    X_list = list()
    Y_list = list()
    Z_list = list()
    atomtype_list = list()
    for strline in strline_L:

        stripped_line = strline.split()

        line_length = len(stripped_line)

        X_list.append(float(stripped_line[0]))
        Y_list.append(float(stripped_line[1]))
        Z_list.append(float(stripped_line[2]))

        atomtype = stripped_line[3]
        if atomtype == 'C':
            atomtype_list.append('h') # 'h' means hydrophobic
        else:
            atomtype_list.append('p') # 'p' means polar

    return X_list, Y_list, Z_list, atomtype_list


In [244]:
######test the model######
import pandas as pd
pred_list = []
for i in range(1,825):
    vox_list = []
    print(i)
    file_name = "{}_{}_cg.pdb".format(("0000" + str(i))[-4:], "pro")
    pro_X_list, pro_Y_list, pro_Z_list, pro_atomtype_list = read_pdb_test('testing_data_release/testing_data/'+file_name)

    #print(len(temp_test_vox_list))
    for j in range(1,825):
        
        lig=j
        file_name = "{}_{}_cg.pdb".format(("0000" + str(lig))[-4:], "lig")
        X_list, Y_list, Z_list, atomtype_list = read_pdb_test('testing_data_release/testing_data/'+file_name)
        x_lower,x_upper,y_lower,y_upper,z_lower,z_upper,vox=create_vox(X_list,Y_list,Z_list,20)
        vox= convert_xyz_to_vox(x_lower,x_upper,y_lower,y_upper,z_lower,z_upper,X_list, Y_list,Z_list,atomtype_list,0,1,vox)
        vox= convert_xyz_to_vox(x_lower,x_upper,y_lower,y_upper,z_lower,z_upper,pro_X_list, pro_Y_list,pro_Z_list,pro_atomtype_list,2,3,vox)
        vox_list.append(vox)
        
    X_test = np.array(vox_list)
    X_test=np.swapaxes(X_test,1,4)
    X_test=np.swapaxes(X_test,2,4)
    X_test=np.swapaxes(X_test,3,4)              
    X_test = torch.from_numpy(X_test).float()
    test_loader = torch.utils.data.DataLoader(X_test, batch_size = 1, shuffle=False, num_workers=4)
    
    model = Net().cuda()
    model.load_state_dict(torch.load("model_project.h5"))
    model.eval()

    prediction = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            outputs = model(data)
            softmax=torch.nn.Softmax()
            outputs=softmax(outputs.data.detach().cpu().numpy()[:,1])
            prediction.append(outputs)

            
    pred_index = pd.DataFrame(prediction, index=np.arange(1,len(prediction)+1)).sort_values(by = [0], ascending =False).head(10).index
    pred_list.append(pred_index)


1
2
3
4
5
6
7
8
9
10


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000245BFC8F318>
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\envs\newfyp\lib\site-packages\torch\utils\data\dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "D:\ProgramData\Anaconda3\envs\newfyp\lib\site-packages\torch\utils\data\dataloader.py", line 1174, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
2

In [246]:
Final_ouput = pd.DataFrame(pred_list, index=np.arange(1,825), columns=['lig1_id','lig2_id','lig3_id','lig4_id','lig5_id','lig6_id','lig7_id','lig8_id','lig9_id','lig10_id'])
Final_ouput.index.names = ['pro_id']

In [247]:
Final_ouput.head(10)

,lig1_id,lig2_id,lig3_id,lig4_id,lig5_id,lig6_id,lig7_id,lig8_id,lig9_id,lig10_id
pro_id,,,,,,,,,,
1,388,664,78,156,731,468,213,766,151,489
2,651,285,85,148,521,367,777,334,315,747
3,593,43,479,20,3,638,543,60,241,777
4,701,649,46,666,120,674,421,360,144,227
5,90,85,222,91,624,505,204,815,194,488
6,714,137,407,79,576,606,404,583,202,661
7,454,426,138,75,368,665,411,335,667,768
8,172,589,3,191,544,243,30,690,457,577
9,790,362,20,565,746,584,450,591,359,170


In [248]:
Final_ouput.to_csv('final_output.txt',sep='\t',index = True)